In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K


from keras.utils import multi_gpu_model
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import tensorflow as tf

import gc

import time

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True

hdf5_file = h5py.File(hdf5_path, "r")

if subtract_mean:
    mm = hdf5_file["train_mean"][...,0]
    mm = mm[np.newaxis, ...]

data_num = hdf5_file["train_flow"].shape[0]
    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

x_train = hdf5_file["train_flow"][...,0]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:,...]
hdf5_file.close()
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")


x_test = hdf5_file["IoT_flow"][...,0]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:,...]

hdf5_file.close()

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

#define number of packets used to train for each device
packets = 298
x_train = x_train[:,:packets,:]
x_test = x_test[:,:packets,:]

input_shape = (x_train.shape[1], x_train.shape[2])
    
    
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
AccScores = []
Times = []
LossScores = []
#AucScores = []
#F1Scores = []
#PrecisionScores = []
t2 = time.time()

for i in range (1,50):
    t
    size = 64
    nb_filters = 32
    num_classes = 2
    epochs=30
    activations= "tanh"
    maxlen=packets
    dropout = 0.24295335733172563
    batch_size = 512
    pool_size = 3
    lr = 0.0012168203727788032
    adam = keras.optimizers.Adam(lr=lr)

    optim = adam

    layers = 3
    model = Sequential()
    model.add(Conv1D(filters=nb_filters,kernel_size=size,input_shape=(input_shape),padding="valid",activation=activations,strides=1))
    for i in range(layers-1):
        model.add(Conv1D(filters=nb_filters,kernel_size=size,padding="valid",activation=activations,strides=1))
        
    model.add(MaxPooling1D(pool_size= pool_size))
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
        
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    t0 = time.time()
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    t1 = time.time()
    total_run = t1-t0
    Times.append(total_run)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.5f%%" % (model.metrics_names[0], scores[0]))
    print('Fit and Evaluate Time:',total_run)
    AccScores.append(scores[1] * 100)
    LossScores.append(scores[0])
    
    #y_pred = model.predict(x_test)
    #yy_test = [np.argmax(i) for i in y_test]

    #yy_pred = [np.argmax(i) for i in y_pred]
  
    #new = np.vstack([yy_test,yy_pred])
    #print(classification_report(yy_test, yy_pred, digits = 5 ))
    #f1 = f1_score(yy_test, y_pred)
    #F1Scores.append(f1)
    
    #ap = average_precision_score(yy_test, y_pred[:,1])
    #PrecisionScores.append(ap)
    

    """

    fpr_keras, tpr_keras, thresholds_keras = roc_curve(yy_test, y_pred[:,0],pos_label=0)
    auc_keras = auc(fpr_keras, tpr_keras)
    AucScores.append(auc_keras)
    """
    del model
    gc.collect()
    K.clear_session()
    gc.collect()

    
print("%.5f%% (+/- %.5f%%)" % (np.mean(AccScores), np.std(AccScores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(AucScores), np.std(AucScores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(F1Scores), np.std(F1Scores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(PrecisionScores), np.std(PrecisionScores)))
t3 = time.time()
total_time = t3-t2
print('Time to do everything:',total_time)


SyntaxError: can't assign to literal (<ipython-input-3-b857f2bcd055>, line 45)

In [ ]:
AccScores

In [ ]:
LossScores

In [ ]:
Times